# Исследование надежности заемщиков.



**Цель исследования** - проверить гипотезы:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок;
2. Есть ли зависимость между семейным положением и возвратом кредита в срок;
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок;
4. Как разные цели кредита влияют на его возврат в срок.

**Ход исследования**

Исследование пройдет в три этапа:
1. Обзор данных;
2. Предобработка данных;
3. Проверка гипотез;
4. Общий вывод.



## 1. Обзор данных

Импортируем бибилотеку `pandas`.

In [1]:
import pandas as pd

Читаем файл `data.csv`, он сохранен в той же папке, что и проект, и сохраняем его в переменной `df`.

In [2]:
df = pd.read_csv('data.csv')

Выводим на экран первые 10 строк таблицы:

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получаем общую информацию о таблице:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов.
Согласно документации к данным:
* `children` - количество детей в семье;
* `days_employed` - общий трудовой стаж в днях;
* `dob_years` - возраст клиента в годах;
* `education` - уровень образования клиента;
* `education_id` - идентификатор уровня образования;
* `family_status` - семейное положение;
* `family_status_id` - идентификатор семейного положения;
* `gender` - пол клиента;
* `income_type` - тип занятости;
* `debt` - имел ли задолженность по возврату кредитов;
* `total_income` - ежемесячный доход;
* `purpose` - цель получения кредита.

В данных замечено три ошибки: 
1. Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.
2. Встречаются аномалии (артефакты) - значения, которые не отражают действительность и появились по какой-то ошибке. Например, отрицательное количество дней трудового стажа в столбце days_employed. 
3. Присутствуют строки-дубликаты и неявные дубликаты. Например, в столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв.

**Выводы**

В каждой строке таблицы — данные о клиенте. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## 2. Предобработка данных

### 2.1 Пропуски

Сначала посчитаем, сколько в таблице пропущенных значений.

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах `days_employed` и `total_income` замечены пропущенные значения. При том, они пропущены в одних и тех же строках. 
Проверим, какую долю составляют пропущенные значения в cтолбцах с пропусками: 

In [6]:
df['days_employed'].isna().sum() / df['children'].count()

0.10099883855981417

Количество пропущенных значений составляет 10%. 

### 2.2 Проверка данных на аномалии и исправления

В данных встречаются артефакты (отрицательное количество дней трудового стажа и детей в столбцах `days_employed` `children`). Изменим их, перезаписав в строки модули значений:

In [7]:
df['days_employed'] = df['days_employed'].abs()
df['children'] = df['children'].abs()

Заменим пропуски в столбцах `days_employed` и `total_income`. Так как данные являются количественными - заполняем характерными значениями (значения, характеризующие состояние выборки). В данном случае, так как разброс чисел велик, воспользуемся медианными значениями. 

In [8]:
days_employed_median = df['days_employed'].median() #рассчитываем медиану
total_income_median = df['total_income'].median()

df['days_employed'] = df['days_employed'].fillna(days_employed_median) #заполняем пропуски медианой
df['total_income'] = df['total_income'].fillna(total_income_median)

Теперь посмотрим на столбец `children`, так как в будущем он нам понадобится для анализа:

In [9]:
df['children'].value_counts() #выводим все значения столбца

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

В данных замечено рекордное количество детей - 20. Скорее всего это ошибка, которая возникла при невнимательном заполнении анкеты. Заменим строчки с такм количеством детей на количество "2"

In [10]:
df['children'] = df['children'].replace(20,2) #выполняем замену
df['children'].value_counts() #проверяем результат


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### 2.3. Изменение типов данных

Для удобного анализа заменяем вещественный тип данных в столбце `total_income` на целочисленный:

In [11]:
df['total_income'] = df['total_income'].astype('int')

### 2.4. Удаление дубликатов

Для начала посчитаем количество явных дубликатов:

In [12]:
df.duplicated().sum() #считаем количество дубликатов

54

Теперь займемся неявными дубликаты, такими как в столбце `education`:

In [13]:
df['education'].unique() #выводим уникальные значение столбца

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Приведем данные к одному регистру:

In [14]:
df['education'] = df.education.str.lower() 
df['education'].unique() #проверяем результат

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Удалим все дубликаты и проверим результат:

In [15]:

df = df.drop_duplicates().reset_index(drop=True) #удаляем дубликаты
df.duplicated().sum()

0

### 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

Создаем два новых датафрейма со столбцами: `education_id` и `education` - в певром, `family_status_id` и `family_status` - во втором. Новые датафреймы — это те самые «словари» , к которым мы сможем обращаться по идентификатору.

In [16]:
education_df = pd.DataFrame(data=df, columns=['education_id', 'education'])
family_df = pd.DataFrame(data=df, columns=['family_status_id', 'family_status'])


Убираем из исходной таблицы `df` столбцы `education` и `family_status`:

In [17]:
df = df.loc[:, ['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income','purpose']]
df.head(10)


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


### 2.6 Категоризация дохода

Для категоризации дохода создаем новый столбец `total_income_category`, со следующими категориями:
* `0–30000 — 'E'`;
* `30001–50000 — 'D'`;
* `50001–200000 — 'C'`;
* `200001–1000000 — 'B'`;
* `1000001 и выше — 'A'`.

In [18]:
def total_income_category(total_income): 
    '''Функция создает буквенные обозначения категорий дохода'''
    if total_income <= 30000:
        return 'E'
    if (total_income > 30000) & (total_income <= 50000):
        return 'D'
    if (total_income > 50000) & (total_income <= 200000):
        return 'C'
    if (total_income > 200000) & (total_income <= 1000000):
        return 'B'
    return 'A'
    


In [19]:
df['total_income_category'] = df['total_income'].apply(total_income_category) #создаем столбец с обозначениями дохода из функции
df['total_income_category'].value_counts() #выводим на экран получившиеся значения

C    16016
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### 2.7 Категоризация целей кредита

Для начала выводим уникальные значения из столбца `purpose`, чтобы сформировать категории:

In [20]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создаем функцию для категоризации со следующими категориями:
* `операции с автомобилем`;
* `операции с недвижимостью`;
* `проведение свадьбы`;
* `получение образования`.

In [21]:
def purpose_category(purpose):
    if ('жиль' or 'недвиж') in purpose:
        return 'операции с недвижимостью'
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образ' in purpose:
        return 'получение образования'

    



Также создаем новый столбец `purpose_category` с категориями:

In [22]:
df['purpose_category'] = df['purpose'].apply(purpose_category) 
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


**Выводы**

Предобработка обнаружила три проблемы в данных:

- аномалии
- пропущенные значения
- дубликаты — явные и неявные

Теперь можно перейти к проверке гипотез. 

## 3. Проверка гипотез

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Построим сводную таблицу, посчитав в ней три агрегирующие функции - количество, сумму и среднее.
`количество (count)` - всего клиентов в группе;
`сумма (sum)` - количество должников;
`среднее (mean)` - доля должников.

In [23]:
hypothesis_first = pd.pivot_table(df, index='children', values='debt', aggfunc=['count', 'sum', 'mean']) #строим сводную таблицу
hypothesis_first.columns = ['total', 'quantity', 'fraction'] #переименуем столбцы
hypothesis_first #выводим результат

,total,quantity,fraction
children,,,
0,14091,1063,0.075438
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


Сформируем вывод.

##### Вывод 1:

У нас задолженность представлена в числах, 1 - есть задолженность, 0 - нет задолженности. Это значит, что среднее арифметическое - это и будет доля должников.

Можно сделать вывод, что больше всего задолжностей по кредитам имеют люди без детей, а меньше - с детьми. Можно предположить, что люди с детьми более отвественны и имеют постоянный доход для погашения кредита.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Получим идентификаторы из созданного датафрейма `family_df`:

In [24]:
family_df.groupby('family_status_id').sum()

,family_status
family_status_id,
0,женат / замужемженат / замужемженат / замужемж...
1,гражданский бракгражданский бракгражданский бр...
2,вдовец / вдовавдовец / вдовавдовец / вдовавдов...
3,в разводев разводев разводев разводев разводев...
4,Не женат / не замужемНе женат / не замужемНе ж...


По тому же принципу строим сводную таблицу для данных из`family_df`:

In [25]:
hypothesis_second = pd.pivot_table(df, index='family_status_id', values='debt', aggfunc=['count', 'sum', 'mean']) #строим сводную таблицу
hypothesis_second.columns = ['total', 'quantity', 'fraction'] #переименуем столбцы
hypothesis_second #выводим результат

,total,quantity,fraction
family_status_id,,,
0,12339,931,0.075452
1,4151,388,0.093471
2,959,63,0.065693
3,1195,85,0.071130
4,2810,274,0.097509


Сформируем вывод.

##### Вывод 2 :

Самую большую долю задолженности имеют клиенты с идентификатором "4" - "Не женат"  и "1" - "гражданский брак".


##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Повторяем построение свобдной таблицы для столбца `total_income_category`:

In [26]:
hypothesis_third = pd.pivot_table(df, index='total_income_category', values='debt', aggfunc=['count', 'sum', 'mean']) #строим сводную таблицу
hypothesis_third.columns = ['total', 'quantity', 'fraction'] #переименуем столбцы
hypothesis_third #выводим результат

,total,quantity,fraction
total_income_category,,,
A,25,2,0.080000
B,5041,356,0.070621
C,16016,1360,0.084915
D,350,21,0.060000
E,22,2,0.090909


Сформируем вывод.

##### Вывод 3:


Самую большую долю задолженности имеют клиенты категории `'Е' (0–30000`) и `'C' (50001–200000)`.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Вновь строим сводную таблицу по столбцу `purpose_category`:

In [27]:
hypothesis_fourth = pd.pivot_table(df, index='purpose_category', values='debt', aggfunc=['count', 'sum', 'mean']) #строим сводную таблицу
hypothesis_fourth.columns = ['total', 'quantity', 'fraction'] #переименуем столбцы
hypothesis_fourth #выводим результат

,total,quantity,fraction
purpose_category,,,
операции с автомобилем,4306,403,0.093590
операции с недвижимостью,4460,308,0.069058
получение образования,4013,370,0.092200
проведение свадьбы,2324,186,0.080034


Сформируем вывод.

##### Вывод 4 :

Больше всего задолженностей по кредитам у людей с целью `получение образования` и `операции с автомобилем`. Это может быть связано с большими и неопределенными (дополнительными) тратами. Так, клиент, взявший кредит на покупку поддержанного автомобиля может потратить дополнительные суммы на его ремонт.



## Общий вывод

В этой работе было проведено исследование надежности заемщиков. 
Сначала данные были обработаны: исключены пропущенные значения, аномалии и строки-дубликаты.

Далее перешли к проверке гипотез: есть ли зависимость между количеством детей и возвратом кредита в срок; между семейным положением и возвратом кредита в срок; между уровнем дохода и возвратом кредита в срок; как разные цели кредита влияют на его возврат в срок?
Гипотезы подтвердились: 
Больше всего задолжностей по кредитам имеют люди без детей, а меньше - с детьми. Можно предположить, что люди с детьми более отвественны и имеют постоянный доход для погашения кредита.
Самую большую долю задолженности имеют клиенты с идентификатором "4" - "Не женат" и "1" - "гражданский брак".
Самую большую долю задолженности имеют клиенты категории 'Е' (0–30000) и 'C' (50001–200000).
Больше всего задолженностей по кредитам у людей с целью получение образования и операции с автомобилем. Это может быть связано с большими и неопределенными (дополнительными) тратами. Так, клиент, взявший кредит на покупку поддержанного автомобиля может потратить дополнительные суммы на его ремонт.